In [5]:
# coding: utf-8

# # 说明
# 双层lstm
# rmse = 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import tensorflow as tf
from numpy.random import RandomState

from keras.models import Sequential
from keras.layers import Dense,Flatten,CuDNNLSTM,Dropout,Activation,Bidirectional
from keras.layers import Conv1D, Convolution1D, MaxPooling1D
from keras.callbacks import ReduceLROnPlateau  #学习率自动变化
from keras.callbacks import EarlyStopping
#from keras.utils import multi_gpu_model

#使用GPU
import keras.backend.tensorflow_backend as KTF 
KTF.set_session(tf.Session(config=tf.ConfigProto(device_count={'gpu':0})))

import time
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA

#matplotlib inline
plt.rcParams['figure.figsize'] = 16,6
plt.rcParams['xtick.color'] = 'w'  
plt.rcParams['ytick.color'] = 'w'  
mpl.style.use('ggplot')

#主要的超参数
sequence_length = 110
unit_lstm1 = 300
unit_lstm2 = 300
dense1 = 50
dropout_rate1 = 0.5
dropout_rate2 = 0.4
dropout_rate3 = 0.4
epoch_num = 55
batch_size_num = 5

# ## 数据导入和处理


train_df_path = r'D:\git\gasturbine-rul-prediction\data\processed_data\train_df.csv'
test_df_path = r'D:\git\gasturbine-rul-prediction\data\processed_data\test_df.csv'

train_df = pd.read_csv(train_df_path,index_col=0) #第一列作为index
test_df = pd.read_csv(test_df_path,index_col=0)


#print("train_df shape: {}".format(train_df.shape))
#print("test_df shape: {}".format(test_df.shape))


# ## 定义X_train, y_train, X_test, y_train


X_train = train_df.iloc[:,5:26]
y_train = train_df.iloc[:,-2]

X_train = np.array(X_train)
y_train = np.array(y_train)

X_test = test_df.iloc[:,5:26]
y_test = test_df.iloc[:,-1]#注意，test_df中RUL列是最后一列，
                           #但是，测试集的RUL不是test_df的RUL对应的列

X_test = np.array(X_test)
y_test = np.array(y_test)

#print("X_train.shape: {}, y_train.shape: {}, X_test.shape: {}, y_test.shape: {}".format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))


# ## 采用时间窗分割的方式改变数据的维度


# 将数据格式变为(样本循环次数, 时间窗大小：30, 特征数)
def gen_sequence(id_df, seq_length, seq_cols):
    data_array = id_df[seq_cols].values
    num_elements = data_array.shape[0]
    for start, stop in zip(
            range(0, num_elements - seq_length), range(seq_length,
                                                       num_elements)):
        yield data_array[start:stop, :]


# 选择特征列
sensor_cols = ['s' + str(i) for i in range(1, 22)]
sequence_cols = ['setting1', 'setting2', 'setting3', 'cycle_norm']
sequence_cols.extend(sensor_cols)

# seq_array为用上函数生成的数组，其形状为(15631, 30, 25)
seq_gen = (list(
    gen_sequence(train_df[train_df['id'] == id], sequence_length,
                 sequence_cols)) for id in train_df['id'].unique())
seq_array = np.concatenate(list(seq_gen)).astype(np.float32)


# 对应数据格式生成标签
def gen_labels(id_df, seq_length, label):
    data_array = id_df[label].values
    num_elements = data_array.shape[0]
    return data_array[seq_length:num_elements, :]


# 标签的形状为(15631, 1)
label_gen = [
    gen_labels(train_df[train_df['id'] == id], sequence_length, ['RUL'])
    for id in train_df['id'].unique()
]
label_array = np.concatenate(label_gen).astype(np.float32)

# 生成test数据的最后一个序列，形状为(93, 50, 25)，不足100是因为有些测试集小于50
seq_array_test_last = [
    test_df[test_df['id'] == id][sequence_cols].values[-sequence_length:]
    for id in test_df['id'].unique()
    if len(test_df[test_df['id'] == id]) >= sequence_length
]
seq_array_test_last = np.asarray(seq_array_test_last).astype(np.float32)

# 对应生成test的label，形状为(93, 1)
y_mask = [
    len(test_df[test_df['id'] == id]) >= sequence_length
    for id in test_df['id'].unique()
]
label_array_test_last = test_df.groupby('id')['RUL'].nth(-1)[y_mask].values
label_array_test_last = label_array_test_last.reshape(
    label_array_test_last.shape[0], 1).astype(np.float32)

#nb_features = seq_array.shape[2]
# nb_features == 25
#nb_out = label_array.shape[1]
# nb_out ==1

# print("seq_array shape: {}".format(seq_array.shape))
# print("label_array shape: {}".format(label_array.shape))
# print("seq_array_test_last shape: {}".format(seq_array_test_last.shape))
# print("label_array_test_last shape: {}".format(label_array_test_last.shape))


X_train = seq_array
y_train = label_array
X_test = seq_array_test_last
y_test = label_array_test_last

print("X_train.shape: {}, y_train.shape: {}, X_test.shape: {}, y_test.shape: {}".format(X_train.shape, y_train.shape, X_test.shape, y_test.shape))

# 现在X_train, y_train, X_test, y_test已经准备好了



X_train.shape: (9631, 110, 25), y_train.shape: (9631, 1), X_test.shape: (67, 110, 25), y_test.shape: (67, 1)


In [4]:
####模型构建#################################

# 我们可以看到LSTM（）层必须指定输入的形状。而且每个LSTM层的输入必须是三维的。这输入的三个维度是：
# 
# - samples。一个序列是一个样本。批次由一个或多个样本组成。
# 
# - timesteps。一个时间步代表样本中的一个观察点。timesteps可以理解为循环神经网络认为每个输入数据与前多少个连续输入的数据有联系
# 
# - features。一个特征是在一个时间步长的观察得到的。
# - 学习率。

model = Sequential()
model.add(Conv1D(16, 1, activation='relu', input_shape=(sequence_length, 25)))
model.add(MaxPooling1D(1))
model.add(Conv1D(32, 1, activation='relu', input_shape=(sequence_length, 25)))
model.add(MaxPooling1D(1))
model.add(
    Bidirectional(CuDNNLSTM(
        unit_lstm1,
        input_shape=(X_train.shape[1], X_train.shape[2]),
        return_sequences=True)))
model.add(Activation('tanh'))
model.add(Dropout(dropout_rate1))

model.add(Bidirectional(CuDNNLSTM(unit_lstm2,return_sequences=False)))
model.add(Activation('tanh'))
model.add(Dropout(dropout_rate2))

model.add(Dense(dense1))
model.add(Activation('tanh')) #
model.add(Dropout(dropout_rate3))

model.add(Dense(1))  
model.add(Activation('linear'))

#在2快GPU上复制模型
#parallel_model = multi_gpu_model(model, gpus=2)

model.compile(loss='mse', optimizer='adam')#rmsprop和adam差别不大
#parallel_model.compile(loss='mse', optimizer='adam')

# ## 模型训练

# 可以调节的地方：
# - 训练时是否shuffle
# - LSTM神经元个数
# - timesteps
# - LSTM层数
# - batchsize
# - K折交叉验证


#patience个epoch loss不下降，就降低学习率
reduce_lr = ReduceLROnPlateau(monitor='val_loss', patience=8,verbose=1,mode='auto')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

start_time = time.clock()
history = model.fit(
    X_train,
    y_train,
    epochs=epoch_num,
    batch_size=batch_size_num, 
    validation_split=0.3,
    callbacks=[reduce_lr,early_stopping],
    verbose=2,
    shuffle=True)  
end_time = time.clock()
print("Training time: {:.4} minutes".format((end_time - start_time) / 60))

'''
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val') 
plt.legend()  
plt.show()
'''

# ## 模型评估

y_pred = model.predict(X_test)


def rmse(predictions,targets):
    return np.sqrt(((predictions-targets)**2).mean())


my_rmse = rmse(y_pred,y_test)


print("*****参数***************************")
print("time_step: {}".format(sequence_length))
print("number of Conv1D is 3")
print("number of filter is 16,32,64")
print("length of filter is 1")
print("unit_lstm1: {}".format(unit_lstm1))
print("unit_lstm2: {}".format(unit_lstm2))
print("dense1: {}".format(dense1))
print("dropout_rate1 :{}".format(dropout_rate1))
print("dropout_rate2 :{}".format(dropout_rate2))
print("dropout_rate3 :{}".format(dropout_rate3))
print("set epoch_num :{}".format(epoch_num))
print("batch_size_num :{}".format(batch_size_num))
print("*****结果***************************")
print("Train rmse: {}".format(np.sqrt(history.history['loss'][-1])))
print("Validation rmse: {}".format(np.sqrt(history.history['val_loss'][-1])))
print("Test rmse: {}".format(my_rmse))
print("***********************************************************")

KeyboardInterrupt: 